# FastF1 Telemetry

### This file contains scripts that serve as ETL footprints for the VisionF1 telemetry data pipeline. They obtain data from the API or the local cache, process it as needed, and the objective is to then upload it to the VisionF1 database (MongoDB Atlas).

## Race Pace

All data from 2018 to now.

In [ ]:
import fastf1
import pandas as pd
from datetime import datetime
from mongodb_utils import prepare_documents, upsert_to_mongo

SEASONS = range(2025, datetime.now().year + 1)

all_race_pace = []

for season in SEASONS:
    schedule = fastf1.get_event_schedule(season)
    schedule = schedule[schedule["EventFormat"] != "testing"]

    for _, event in schedule.iterrows():
        event_name = event["EventName"]
        round_number = event["RoundNumber"]
        try:
            session = fastf1.get_session(season, event_name, 'R')
            session.load()
        except Exception as e:
            print(f"Error loading session {season} {event_name}.")
            continue

        if not hasattr(session, '_laps'):
            print(f"No lap data for {season} {event_name}, skipping.")
            continue
        
        laps = session.laps
        laps = laps[laps['IsAccurate']]
        drivers = laps['Driver'].unique()

        for driver in drivers:
            filtered_laps = laps.pick_drivers([driver])
            
            avg_laptime = filtered_laps["LapTime"].mean()
            std_laptime = filtered_laps["LapTime"].std()

            all_race_pace.append({
                "season": season,
                "round": int(round_number),
                "event": event_name,
                "driver": driver,
                "avg_laptime": avg_laptime.total_seconds(),
                "std_laptime": std_laptime.total_seconds() if pd.notnull(std_laptime) else None,
                "race_pace_id": f"{season}_{round_number}_{driver}"
            })

df_race_pace = pd.DataFrame(all_race_pace)

In [33]:
df_race_pace

,season,round,event,driver,avg_laptime,std_laptime,race_pace_id
0,2025,1,Australian Grand Prix,VER,90.314777,2.618743,2025_1_VER
1,2025,1,Australian Grand Prix,GAS,92.140342,2.566940,2025_1_GAS
2,2025,1,Australian Grand Prix,ANT,91.734382,2.871252,2025_1_ANT
3,2025,1,Australian Grand Prix,ALO,92.502320,2.265482,2025_1_ALO
4,2025,1,Australian Grand Prix,LEC,91.704388,2.618302,2025_1_LEC
...,...,...,...,...,...,...,...
340,2025,18,Singapore Grand Prix,COL,98.863120,1.176053,2025_18_COL
341,2025,18,Singapore Grand Prix,BOR,98.736327,1.050940,2025_18_BOR
342,2025,18,Singapore Grand Prix,OCO,98.871396,1.067156,2025_18_OCO
343,2025,18,Singapore Grand Prix,GAS,98.458625,1.555571,2025_18_GAS


In [35]:
ver = df_race_pace[ df_race_pace["driver"] == "VER" ]
ver_2025 = ver[ ver["season"] == 2025 ]
ver_2025

,season,round,event,driver,avg_laptime,std_laptime,race_pace_id
0,2025,1,Australian Grand Prix,VER,90.314777,2.618743,2025_1_VER
20,2025,2,Chinese Grand Prix,VER,97.199452,1.050277,2025_2_VER
37,2025,3,Japanese Grand Prix,VER,92.459200,0.927919,2025_3_VER
62,2025,4,Bahrain Grand Prix,VER,98.083375,0.960517,2025_4_VER
78,2025,5,Saudi Arabian Grand Prix,VER,93.191066,0.582681,2025_5_VER
95,2025,6,Miami Grand Prix,VER,91.751291,1.136319,2025_6_VER
114,2025,7,Emilia Romagna Grand Prix,VER,80.305176,1.168307,2025_7_VER
134,2025,8,Monaco Grand Prix,VER,75.852114,1.453537,2025_8_VER
154,2025,9,Spanish Grand Prix,VER,79.654830,1.071211,2025_9_VER
173,2025,10,Canadian Grand Prix,VER,75.547278,0.688125,2025_10_VER


In [ ]:
schedule = fastf1.get_event_schedule(2025)
schedule

In [25]:
#season = 2025
#event_name = "Azerbaijan Grand Prix"
season = 2024
event_name = 'United States Grand Prix'
round_number = 17
session = fastf1.get_session(season, event_name, 'R')
session.load()
laps = session.laps
laps = laps[laps['IsAccurate']]
drivers = laps['Driver'].unique()

all_race_pace = []

for driver in drivers:
    filtered_laps = laps.pick_drivers([driver])

    avg_laptime = filtered_laps["LapTime"].mean()
    std_laptime = filtered_laps["LapTime"].std()

    all_race_pace.append({
        "season": season,
        "round": int(round_number),
        "event": event_name,
        "driver": driver,
        "avg_laptime": avg_laptime.total_seconds(),
        "std_laptime": std_laptime.total_seconds() if pd.notnull(std_laptime) else None,
        "race_pace_id": f"{season}_{round_number}_{driver}"
    })

all_race_pace.sort(key=lambda x: x['avg_laptime'])

df_race_pace = pd.DataFrame(all_race_pace)
df_race_pace

events      WARNING 	Correcting user input 'United States Grand Prix' to 'United States Grand Prix'
2025-10-15 23:07:08,823 WARNING Correcting user input 'United States Grand Prix' to 'United States Grand Prix'
core           INFO 	Loading data for United States Grand Prix - Race [v3.5.3]
2025-10-15 23:07:08,827 INFO Loading data for United States Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
2025-10-15 23:07:08,829 INFO Using cached data for session_info
req            INFO 	Using cached data for driver_info
2025-10-15 23:07:08,831 INFO Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
2025-10-15 23:07:09,877 INFO Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
2025-10-15 23:07:09,886 INFO Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
2025-10-15 23:07:09,889 INFO Using cached data for track_status_data
req    

,season,round,event,driver,avg_laptime,std_laptime,race_pace_id
0,2024,17,United States Grand Prix,LEC,98.941140,0.631499,2024_17_LEC
1,2024,17,United States Grand Prix,SAI,99.108080,0.850878,2024_17_SAI
2,2024,17,United States Grand Prix,NOR,99.203360,0.980651,2024_17_NOR
3,2024,17,United States Grand Prix,VER,99.301780,0.724766,2024_17_VER
4,2024,17,United States Grand Prix,PIA,99.355720,1.125157,2024_17_PIA
5,2024,17,United States Grand Prix,RUS,99.897420,1.557708,2024_17_RUS
6,2024,17,United States Grand Prix,PER,100.076460,1.118916,2024_17_PER
7,2024,17,United States Grand Prix,HUL,100.093760,1.248924,2024_17_HUL
8,2024,17,United States Grand Prix,MAG,100.115500,1.173763,2024_17_MAG
9,2024,17,United States Grand Prix,LAW,100.278800,1.383116,2024_17_LAW
